In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 8)


# TensorFlow ≥2.10 is required
import tensorflow as tf
assert tf.__version__ >= "2.10"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

**Data Fetching and Processing**

In [ ]:
# Get CIFAR100 data from Keras datasets
# In this task, targets are irrelevant. Therefore we just keep the 32*32 RGB images

from tensorflow.keras.datasets import cifar100

(x_trainC, _), (x_testC, _) = cifar100.load_data()

In [ ]:
# Normalize images

x_trainC = x_trainC.astype('float32') / 255.
x_testC = x_testC.astype('float32') / 255.

print(x_trainC.shape)
print(x_testC.shape)

In [ ]:
# Add noise to images.
# The parameter noise_factor defines the strength of the perturbation
# https://www.tensorflow.org/api_docs/python/tf/random/normal

# After the noise perturbation, the values still belong to the interval [0, 1]

noise_factor = 0.1
x_trainC_noisy = x_trainC + noise_factor * tf.random.normal(shape=x_trainC.shape)
x_testC_noisy = x_testC + noise_factor * tf.random.normal(shape=x_testC.shape)

x_trainC_noisy = tf.clip_by_value(x_trainC_noisy, clip_value_min=0., clip_value_max=1.)
x_testC_noisy = tf.clip_by_value(x_testC_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:
# Visualize a few images before and after the perturbation
# Change the value of start to inspect other images

start= 10

n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.title("Original")
    plt.imshow(tf.squeeze(x_testC[i+start]))
    plt.gray()

    ax = plt.subplot(2, n, i + n + 1)
    plt.title("Noisy")
    plt.imshow(tf.squeeze(x_testC_noisy[i+start]))
    plt.gray()

plt.show()

**Creation of the Denoising AutoEncoder**

In [ ]:
# Denoise is a subclass of the generic Model class
# Complete the Decoder section. You should use Conv2DTranspose layers to reconstruct (upsample) the latent representation to the original image dimensions
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose
# https://keras.io/api/models/model/#model-class

from tensorflow.keras.models import Model

class DenoiseC(Model):
  def __init__(self):
    super(DenoiseC, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(32, 32, 3)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2DTranspose(16, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
# Create an Autoencoder object

tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

autoencoderC = DenoiseC()

**Compile and Train**

In [ ]:
# Model compilation
# This is a regression problem, where the loss corresponds to the difference between the input (noisy image) and the output (original image)
# Therefore, the Autoencoder must remove the noise from the input , aiming at delivering a clear image

autoencoderC.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

autoencoderC.fit(x_trainC_noisy, x_trainC,
                epochs=10,
                shuffle=True)

In [ ]:
# Confirm the reduction of dimension from the original image to the latent representation (Encoder)
# Confirm the expansion from the latent representation to the original dimension (Decoder)

autoencoderC.summary()

print('***Encoder***')
autoencoderC.encoder.summary()

print('***Decoder***')
autoencoderC.decoder.summary()


**Evaluate Performance**

In [ ]:
# Evaluate performance on the test set

autoencoderC.evaluate(x_testC_noisy, x_testC)

In [ ]:
# Apply trained autoencoder to images from the test set

encoded_imgs = autoencoderC.encoder(x_testC_noisy).numpy()
decoded_imgs = autoencoderC.decoder(encoded_imgs).numpy()

In [ ]:
# Visualize all transformations
# 1. Original images
# 2. Noise images
# 3. Denoised images


n = 10
plt.figure(figsize=(20, 10))
for i in range(n):

    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.title("Original")
    plt.imshow(tf.squeeze(x_testC[i+start]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display original + noise
    ax = plt.subplot(3, n, i + n + 1)
    plt.title("Noisy")
    plt.imshow(tf.squeeze(x_testC_noisy[i+start]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(3, n, i + n*2 + 1)
    plt.title("Denoised")
    plt.imshow(tf.squeeze(decoded_imgs[i+start]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()
